In [118]:
import requests
import pandas as pd
import itertools

In [120]:
# Replace with your IEX Cloud API token
api_token = 'pk_b23ba153ee2045aeb5ca77fe63ab5cb0'

# Define the base URL for the IEX Cloud API
base_url = 'https://cloud.iexapis.com/stable'

# Define the symbol for SPY
symbol = 'SPY'

# Define the date as May 10, 2024
yesterday_date = '20240510'

# Define the historical endpoint
endpoint = f'/stock/{symbol}/chart/date/{yesterday_date}'

# Set parameters
params = {
    'token': api_token,
}

# Make the API request
response = requests.get(f'{base_url}{endpoint}', params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Check if data is not empty
    if data:
        # Convert the data to a DataFrame
        df = pd.DataFrame(data)
    else:
        print("No data available for yesterday.")
else:
    print(f"Error: {response.status_code}, {response.text}")

In [122]:
# Data preparation
df=df[['minute','open','close']].dropna(subset=['open', 'close'])

# Re-indexing
df.index = range(1, len(df) + 1)

# Limiting data sample
df=df.head(100)

In [124]:
# Function to calculate profit for a given buying and selling range
def calculate_profit(df, buy_indices, sell_indices):
    profit = 0
    for buy_index in buy_indices:
        for sell_index in sell_indices:
            profit += df.loc[sell_index]['close'] - df.loc[buy_index]['open']
    return profit

# Convert 'minute' column to index
df.set_index('minute', inplace=True)

# Initialize variables to track best strategy and maximum profit
best_strategy = []
max_profit = float('-inf')

# Iterate through all combinations of buying and selling points
for buy_indices in itertools.combinations(df.index, 2):
    for sell_indices in itertools.combinations(df.index, 2):
        current_profit = calculate_profit(df, buy_indices, sell_indices)
        if current_profit > max_profit:
            best_strategy = (buy_indices, sell_indices)
            max_profit = current_profit

print("Best buy strategy:", best_strategy)
print("Maximum profit:", max_profit)

KeyboardInterrupt: 